<a href="https://colab.research.google.com/github/cristhiansito123/MLDS6project/blob/entrega4/scripts/training/DefinitiveModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


import neccesary packages

In [2]:
import json
import requests
import os
import shutil
import time
import sys

import numpy as np
import pandas as pd

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
%matplotlib inline

#Carga de datos

In [4]:

metadata_path = '/content/drive/MyDrive/MLDS6project/data.csv'

metadata = pd.read_csv(metadata_path, sep=',')


In [5]:
metadata

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


# Modelo

In [6]:
mapeo = {'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5}

# Aplicar el mapeo a la columna del DataFrame
metadata['type_numeric'] = metadata['type'].map(mapeo)

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from sklearn.svm import OneClassSVM


In [8]:
feature_names = ['step', 'type_numeric', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest','isFraud','isFlaggedFraud']
X = metadata[feature_names]

In [9]:
X_def = X.loc[X['isFraud']==0]
X_fraud = X.loc[X['isFraud']==1]

In [10]:
X_def

,step,type_numeric,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,2,9839.64,170136.0,160296.36,0.00,0.00,0,0
1,1,2,1864.28,21249.0,19384.72,0.00,0.00,0,0
4,1,2,11668.14,41554.0,29885.86,0.00,0.00,0,0
5,1,2,7817.71,53860.0,46042.29,0.00,0.00,0,0
6,1,2,7107.77,183195.0,176087.23,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...
6362319,718,2,8634.29,518802.0,510167.71,0.00,0.00,0,0
6362320,718,1,159188.22,3859.0,0.00,0.00,159188.22,0,0
6362321,718,1,186273.84,168046.0,0.00,24893.67,211167.51,0,0
6362322,718,4,82096.45,13492.0,0.00,0.00,82096.45,0,0


In [11]:
X_fraud

,step,type_numeric,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,4,181.00,181.00,0.0,0.00,0.00,1,0
3,1,1,181.00,181.00,0.0,21182.00,0.00,1,0
251,1,4,2806.00,2806.00,0.0,0.00,0.00,1,0
252,1,1,2806.00,2806.00,0.0,26202.00,0.00,1,0
680,1,4,20128.00,20128.00,0.0,0.00,0.00,1,0
...,...,...,...,...,...,...,...,...,...
6362615,743,1,339682.13,339682.13,0.0,0.00,339682.13,1,0
6362616,743,4,6311409.28,6311409.28,0.0,0.00,0.00,1,0
6362617,743,1,6311409.28,6311409.28,0.0,68488.84,6379898.11,1,0
6362618,743,4,850002.52,850002.52,0.0,0.00,0.00,1,0


In [12]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test = train_test_split(X_def,  test_size=0.0015)

In [13]:
X_test

,step,type_numeric,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2265447,187,2,7123.69,4201.0,0.00,0.00,0.00,0,0
365993,17,1,316155.78,0.0,0.00,1660089.32,2201210.85,0,0
5767154,400,1,301184.82,6049.0,0.00,134874.49,436059.31,0,0
3002438,232,1,67365.05,11324.0,0.00,66900.02,134265.07,0,0
3944236,286,2,12097.38,0.0,0.00,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...
1884565,164,1,151253.45,49836.0,0.00,331982.85,483236.30,0,0
1349388,137,3,43596.70,49837.0,93433.70,922936.43,879339.73,0,0
1949688,178,2,48113.05,0.0,0.00,0.00,0.00,0,0
2556252,206,3,179975.54,102790.0,282765.54,0.00,0.00,0,0


Se reduce la cantidad de datos

In [22]:
X_train2 = X_train.drop('isFraud', axis=1)

In [26]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
X_resampled_downsized = resample(X_train2, n_samples=500000)

Se crea y entrena el modelo

In [38]:
# Crear una instancia de One-Class SVM
svm = OneClassSVM(kernel='sigmoid', nu=0.0015)  # Ajusta los hiperparámetros según sea necesario

# Entrenar el modelo con los datos de transacciones legítimas
svm.fit(X_resampled_downsized)

OneClassSVM(kernel='sigmoid', nu=0.0015)

Evaluar modelo

In [43]:
y_true = X['isFraud'].to_numpy()

In [44]:
y_true

array([0, 0, 1, ..., 1, 1, 1])

In [45]:
X_test_def2 = X.drop('isFraud', axis=1)

In [46]:
y_pred = svm.predict(X_test_def2)

In [53]:
y_pred_2 = np.where(y_pred == 1, 0, 1)

In [55]:
y_pred_2

array([0, 0, 1, ..., 0, 0, 0])

In [56]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
precision = precision_score(y_true, y_pred_2)
print("Precisión:", precision)

Precisión: 0.001388147357180993


In [57]:
accuracy = accuracy_score(y_true, y_pred_2)
print("Exactitud:", accuracy)

Exactitud: 0.9972413879816805


In [58]:
f1 = f1_score(y_true, y_pred_2)
print("F1-Score:", f1)

F1-Score: 0.0014791216293093639


In [59]:
recall = recall_score(y_true, y_pred_2)
print("Recall:", recall)

Recall: 0.0015828564470960673


In [60]:
confusion = confusion_matrix(y_true, y_pred_2)
print("Matriz de confusión:")
print(confusion)

Matriz de confusión:
[[6345055    9352]
 [   8200      13]]
